# 🧮 Phase 2B — Multi-year Bioeconomic LP with Dynamics
This notebook adds:
1) **Crop storage carryover** (inventory `inv[h,c,t]`)
2) **Livestock herd dynamics** (herd `herd[h,l,t]` and sales `sale_live[h,l,t]`)

Still linear (LP), solved with `scipy.optimize.linprog`.


In [1]:
import json
from dataclasses import dataclass
from typing import Dict
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.optimize import linprog

try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

DATA = Path('./data_phase_2B')
OUT = Path('./outputs'); OUT.mkdir(exist_ok=True, parents=True)
print('Loaded numpy, pandas, scipy.optimize.linprog')


Loaded numpy, pandas, scipy.optimize.linprog


In [2]:
def must(path: Path) -> Path:
    if not path.exists():
        raise FileNotFoundError(f'Missing required file: {path}')
    return path

hh_df = pd.read_csv(must(DATA/'households.csv'))
crops_df = pd.read_csv(must(DATA/'crops.csv'))
livest_df = pd.read_csv(must(DATA/'livestock.csv'))
prices_df = pd.read_csv(must(DATA/'prices.csv'))
scenario = json.load(open(must(DATA/'scenario_multi.json'),'r'))

init_stocks_df = pd.read_csv(must(DATA/'initial_stocks.csv'))
init_herd_df = pd.read_csv(must(DATA/'initial_herd.csv'))

for name, df in [
    ('households.csv', hh_df),
    ('crops.csv', crops_df),
    ('livestock.csv', livest_df),
    ('prices.csv', prices_df),
    ('initial_stocks.csv', init_stocks_df),
    ('initial_herd.csv', init_herd_df),
    ('scenario_multi.json', pd.DataFrame([{'T':scenario.get('T'), 'discount_rate':scenario.get('discount_rate')}])),
]:
    if display_dataframe_to_user: display_dataframe_to_user(name, df)
    else: display(df.head())

print('Loaded inputs.')


,name,n_households,adult_equiv,labor_endowment,land_available,max_hired_labor
0,HFR,100,3.8,220,0.8,120
1,HMR,100,3.9,240,1.0,130
2,MFIR,100,4.2,300,1.2,160
3,MMR,100,4.2,300,1.2,160
4,MMIR,100,4.6,320,1.5,180


,name,calorie_per_kg,yield_per_ha,price_sale,seed_cost_per_ha,fert_cost_per_ha,chem_cost_per_ha,labor_req_per_ha,storage_cost_per_kg,max_storage_kg,spoilage_rate
0,maize,3600,3500,5.0,800,1800,600,60,1,500,0.05
1,wheat,3200,3000,6.0,700,1600,500,50,1,400,0.04
2,rice,4000,4000,7.0,900,2000,700,80,1,600,0.03
3,beans,3400,1500,8.0,900,700,400,70,2,300,0.06


,name,price_sale,feed_cost_per_unit,vet_cost_per_unit,labor_req_per_unit,reproduction_rate,max_herd
0,goat,2500,800,200,6.0,0.10,100
1,chicken,300,60,15,0.6,0.15,200


,wage
0,80


,household_class,crop,stock_kg
0,HFR,maize,0.0


,household_class,livestock,herd_units
0,HFR,goat,0.0


,T,discount_rate
0,10,0.1


Loaded inputs.


In [3]:
@dataclass
class HouseholdClass:
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float

@dataclass
class CropParam:
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float
    storage_cost_per_kg: float
    max_storage_kg: float
    spoilage_rate: float

@dataclass
class LivestockParam:
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float
    reproduction_rate: float
    max_herd: float

@dataclass
class PriceParam:
    wage: float

@dataclass
class ModelParams:
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

def _req(x, field):
    if pd.isna(x) or str(x).strip()=='':
        raise ValueError(f'Missing value for {field}')
    return float(x)

def load_params() -> ModelParams:
    households = {str(r['name']).strip(): HouseholdClass(
        name=str(r['name']).strip(),
        n_households=_req(r['n_households'],'households.n_households'),
        adult_equiv=_req(r['adult_equiv'],'households.adult_equiv'),
        labor_endowment=_req(r['labor_endowment'],'households.labor_endowment'),
        land_available=_req(r['land_available'],'households.land_available'),
        max_hired_labor=_req(r['max_hired_labor'],'households.max_hired_labor'),
    ) for _, r in hh_df.iterrows()}

    crops = {str(r['name']).strip(): CropParam(
        name=str(r['name']).strip(),
        calorie_per_kg=_req(r['calorie_per_kg'],'crops.calorie_per_kg'),
        yield_per_ha=_req(r['yield_per_ha'],'crops.yield_per_ha'),
        price_sale=_req(r['price_sale'],'crops.price_sale'),
        seed_cost_per_ha=_req(r['seed_cost_per_ha'],'crops.seed_cost_per_ha'),
        fert_cost_per_ha=_req(r['fert_cost_per_ha'],'crops.fert_cost_per_ha'),
        chem_cost_per_ha=_req(r['chem_cost_per_ha'],'crops.chem_cost_per_ha'),
        labor_req_per_ha=_req(r['labor_req_per_ha'],'crops.labor_req_per_ha'),
        storage_cost_per_kg=_req(r['storage_cost_per_kg'],'crops.storage_cost_per_kg'),
        max_storage_kg=_req(r['max_storage_kg'],'crops.max_storage_kg'),
        spoilage_rate=_req(r['spoilage_rate'],'crops.spoilage_rate'),
    ) for _, r in crops_df.iterrows()}

    livestock = {str(r['name']).strip(): LivestockParam(
        name=str(r['name']).strip(),
        price_sale=_req(r['price_sale'],'livestock.price_sale'),
        feed_cost_per_unit=_req(r['feed_cost_per_unit'],'livestock.feed_cost_per_unit'),
        vet_cost_per_unit=_req(r['vet_cost_per_unit'],'livestock.vet_cost_per_unit'),
        labor_req_per_unit=_req(r['labor_req_per_unit'],'livestock.labor_req_per_unit'),
        reproduction_rate=_req(r['reproduction_rate'],'livestock.reproduction_rate'),
        max_herd=_req(r['max_herd'],'livestock.max_herd'),
    ) for _, r in livest_df.iterrows()}

    prices = PriceParam(wage=_req(prices_df.iloc[0]['wage'],'prices.wage'))
    return ModelParams(households=households, crops=crops, livestock=livestock, prices=prices)

def _as_T(x, T, name):
    if isinstance(x, (int,float)): return [float(x)]*T
    if isinstance(x, list):
        if len(x)!=T: raise ValueError(f'scenario.{name} must have length T={T}, got {len(x)}')
        return [float(v) for v in x]
    raise ValueError(f'scenario.{name} must be number or length-T list')

def load_scenario(sc):
    T = int(sc.get('T', 10))
    r = float(sc.get('discount_rate', 0.10))
    Y = _as_T(sc.get('yield_multiplier', 1.0), T, 'yield_multiplier')
    P = _as_T(sc.get('crop_price_multiplier', 1.0), T, 'crop_price_multiplier')
    W = _as_T(sc.get('wage_multiplier', 1.0), T, 'wage_multiplier')
    F = _as_T(sc.get('fert_price_multiplier', 1.0), T, 'fert_price_multiplier')
    POP = _as_T(sc.get('population_multiplier', 1.0), T, 'population_multiplier')
    DISC = [1.0/((1.0+r)**t) for t in range(T)]
    return T, r, Y, P, W, F, POP, DISC

params = load_params()
T, r, Ymul, Pmul, Wmul, Fmul, POPmul, DISC = load_scenario(scenario)
print(f'Loaded: H={len(params.households)}, C={len(params.crops)}, L={len(params.livestock)}, T={T}, r={r}')


Loaded: H=6, C=4, L=2, T=10, r=0.1


In [4]:
def build_index(H, C, L, T):
    idx = {}; pos = 0
    for t in range(T):
        for h in H:
            for c in C: idx[('area', h, c, t)] = pos; pos += 1
        for h in H:
            for c in C: idx[('cons', h, c, t)] = pos; pos += 1
        for h in H:
            for c in C: idx[('sold', h, c, t)] = pos; pos += 1
        for h in H:
            for c in C: idx[('inv', h, c, t)] = pos; pos += 1
        for h in H: idx[('hired', h, None, t)] = pos; pos += 1
        for h in H: idx[('off_farm', h, None, t)] = pos; pos += 1
        for h in H:
            for l in L: idx[('herd', h, l, t)] = pos; pos += 1
        for h in H:
            for l in L: idx[('sale_live', h, l, t)] = pos; pos += 1
    return idx, pos

def init_maps(init_stocks_df, init_herd_df, H, C, L):
    init_stock = {(str(r['household_class']).strip(), str(r['crop']).strip()): float(r['stock_kg'])
                  for _, r in init_stocks_df.iterrows()}
    init_herd = {(str(r['household_class']).strip(), str(r['livestock']).strip()): float(r['herd_units'])
                 for _, r in init_herd_df.iterrows()}
    for h in H:
        for c in C: init_stock.setdefault((h,c), 0.0)
        for l in L: init_herd.setdefault((h,l), 0.0)
    return init_stock, init_herd

def solve_phase2B(params: ModelParams, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC):
    H = list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())
    idx, nvars = build_index(H, C, L, T)
    cvec = np.zeros(nvars)
    bounds = [(0, None) for _ in range(nvars)]
    init_stock, init_herd = init_maps(init_stocks_df, init_herd_df, H, C, L)

    for t in range(T):
        disc = DISC[t]
        wage_t = params.prices.wage * Wmul[t]
        for h in H:
            cvec[idx[('hired', h, None, t)]] += disc * wage_t
            cvec[idx[('off_farm', h, None, t)]] += -disc * wage_t
            for cn in C:
                cp = params.crops[cn]
                cvec[idx[('sold', h, cn, t)]] += -disc * (cp.price_sale * Pmul[t])
                per_ha = cp.seed_cost_per_ha + cp.fert_cost_per_ha * Fmul[t] + cp.chem_cost_per_ha
                cvec[idx[('area', h, cn, t)]] += disc * per_ha
                cvec[idx[('inv', h, cn, t)]] += disc * cp.storage_cost_per_kg
            for l in L:
                lv = params.livestock[l]
                cvec[idx[('sale_live', h, l, t)]] += -disc * lv.price_sale
                cvec[idx[('herd', h, l, t)]] += disc * (lv.feed_cost_per_unit + lv.vet_cost_per_unit)

    A_eq, b_eq, A_ub, b_ub = [], [], [], []

    # Inventory dynamics
    for t in range(T):
        for h in H:
            for cn in C:
                cp = params.crops[cn]
                row = np.zeros(nvars)
                row[idx[('inv', h, cn, t)]] = 1.0
                row[idx[('area', h, cn, t)]] = -(cp.yield_per_ha * Ymul[t])
                row[idx[('cons', h, cn, t)]] = 1.0
                row[idx[('sold', h, cn, t)]] = 1.0
                if t == 0:
                    A_eq.append(row); b_eq.append((1.0 - cp.spoilage_rate) * init_stock[(h, cn)])
                else:
                    row[idx[('inv', h, cn, t-1)]] = -(1.0 - cp.spoilage_rate)
                    A_eq.append(row); b_eq.append(0.0)

    # Herd dynamics
    for t in range(T):
        for h in H:
            for l in L:
                lv = params.livestock[l]
                row = np.zeros(nvars)
                row[idx[('herd', h, l, t)]] = 1.0
                row[idx[('sale_live', h, l, t)]] = 1.0
                if t == 0:
                    A_eq.append(row); b_eq.append((1.0 + lv.reproduction_rate) * init_herd[(h, l)])
                else:
                    row[idx[('herd', h, l, t-1)]] = -(1.0 + lv.reproduction_rate)
                    A_eq.append(row); b_eq.append(0.0)

    # Land
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C:
                row[idx[('area', h, cn, t)]] = 1.0
            A_ub.append(row); b_ub.append(params.households[h].land_available)

    # Labor
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C: row[idx[('area', h, cn, t)]] = params.crops[cn].labor_req_per_ha
            for l in L: row[idx[('herd', h, l, t)]] = params.livestock[l].labor_req_per_unit
            row[idx[('off_farm', h, None, t)]] = 1.0
            row[idx[('hired', h, None, t)]] = -1.0
            A_ub.append(row); b_ub.append(params.households[h].labor_endowment)

    # Hired cap
    for t in range(T):
        for h in H:
            row = np.zeros(nvars); row[idx[('hired', h, None, t)]] = 1.0
            A_ub.append(row); b_ub.append(params.households[h].max_hired_labor)

    # Calorie floor
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C: row[idx[('cons', h, cn, t)]] = -params.crops[cn].calorie_per_kg
            kcal_need = params.min_kcal_per_person_per_day * (params.households[h].adult_equiv * POPmul[t]) * params.days_per_year
            A_ub.append(row); b_ub.append(-kcal_need)

    # Storage cap
    for t in range(T):
        for h in H:
            for cn in C:
                cp = params.crops[cn]
                row = np.zeros(nvars); row[idx[('inv', h, cn, t)]] = 1.0
                A_ub.append(row); b_ub.append(cp.max_storage_kg)

    # Herd cap
    for t in range(T):
        for h in H:
            for l in L:
                lv = params.livestock[l]
                if lv.max_herd and lv.max_herd > 0:
                    row = np.zeros(nvars); row[idx[('herd', h, l, t)]] = 1.0
                    A_ub.append(row); b_ub.append(lv.max_herd)

    res = linprog(cvec, A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  bounds=bounds, method='highs')
    return res, idx, H, C, L

res, idx, H, C, L = solve_phase2B(params, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC)
print('Status:', res.message)
print('Success:', bool(res.success))
if not res.success:
    raise RuntimeError('Optimization failed. Check feasibility / coefficients.')

x = res.x
def v(kind, h, k, t): return float(x[idx[(kind, h, k, t)]])

rows=[]
for t in range(T):
    wage_t = params.prices.wage * Wmul[t]
    for h in H:
        rev_crops = sum(v('sold', h, cn, t) * (params.crops[cn].price_sale * Pmul[t]) for cn in C)
        rev_livestock = sum(v('sale_live', h, l, t) * params.livestock[l].price_sale for l in L)
        rev_off = v('off_farm', h, None, t) * wage_t
        cost_crop_inputs = sum(v('area', h, cn, t) * (params.crops[cn].seed_cost_per_ha + params.crops[cn].fert_cost_per_ha * Fmul[t] + params.crops[cn].chem_cost_per_ha) for cn in C)
        cost_storage = sum(v('inv', h, cn, t) * params.crops[cn].storage_cost_per_kg for cn in C)
        cost_livestock = sum(v('herd', h, l, t) * (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) for l in L)
        cost_hired = v('hired', h, None, t) * wage_t
        profit = (rev_crops + rev_livestock + rev_off) - (cost_crop_inputs + cost_storage + cost_livestock + cost_hired)
        rows.append({'year': t+1, 'household_class': h,
                     'rev_crops': rev_crops, 'rev_livestock_sales': rev_livestock, 'rev_off_farm': rev_off,
                     'cost_crop_inputs': cost_crop_inputs, 'cost_storage': cost_storage,
                     'cost_livestock': cost_livestock, 'cost_hired_labor': cost_hired,
                     'profit': profit, 'discount_factor': DISC[t], 'discounted_profit': profit*DISC[t]})
yearly_df = pd.DataFrame(rows)
npv_df = yearly_df.groupby('household_class', as_index=False).agg(NPV=('discounted_profit','sum'))

if display_dataframe_to_user:
    display_dataframe_to_user('Phase 2B — Yearly results', yearly_df)
    display_dataframe_to_user('Phase 2B — NPV summary', npv_df)
else:
    display(yearly_df.head(12)); display(npv_df)

yearly_df.to_csv(OUT/'phase2B_yearly_results.csv', index=False)
npv_df.to_csv(OUT/'phase2B_npv_summary.csv', index=False)
json.dump({'success': True, 'objective_npv': float(-res.fun), 'T': T, 'discount_rate': r}, open(OUT/'phase2B_meta.json','w'), indent=2)
print('Saved outputs in ./outputs/')


Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True


,year,household_class,rev_crops,rev_livestock_sales,rev_off_farm,cost_crop_inputs,cost_storage,cost_livestock,cost_hired_labor,profit,discount_factor,discounted_profit
0,1,HFR,17545.50,0.0,22080.0,2880.0,0.0,0.0,9600.0,27145.50,1.000000,27145.500000
1,1,HMR,23017.75,0.0,23200.0,3600.0,0.0,0.0,10400.0,32217.75,1.000000,32217.750000
2,1,MFIR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,1.000000,40234.500000
3,1,MMR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,1.000000,40234.500000
4,1,MMIR,36123.50,0.0,30400.0,5400.0,0.0,0.0,14400.0,46723.50,1.000000,46723.500000
5,1,MFR,22890.00,0.0,28000.0,3600.0,0.0,0.0,12000.0,35290.00,1.000000,35290.000000
6,2,HFR,17545.50,0.0,22080.0,2880.0,0.0,0.0,9600.0,27145.50,0.909091,24677.727273
7,2,HMR,23017.75,0.0,23200.0,3600.0,0.0,0.0,10400.0,32217.75,0.909091,29288.863636
8,2,MFIR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,0.909091,36576.818182
9,2,MMR,28234.50,0.0,29120.0,4320.0,0.0,0.0,12800.0,40234.50,0.909091,36576.818182


,household_class,NPV
0,HFR,183477.081005
1,HMR,217760.539557
2,MFIR,271945.943736
3,MFR,238525.950476
4,MMIR,315805.249280
5,MMR,271945.943736


Saved outputs in ./outputs/
